In [ ]:
# !pip install --upgrade transformers

In [1]:
from google.colab import files
uploaded = files.upload()

Saving legal_ner_sample.csv to legal_ner_sample.csv


In [3]:
import pandas as pd

df = pd.read_csv("legal_ner_sample.csv")
df.head()

,sentence,entity,label
0,"This contract is entered into on January 1, 20...","January 1, 2022",DATE
1,"This contract is entered into on January 1, 20...",Alpha Corp,ORG
2,"This contract is entered into on January 1, 20...",Beta LLC,ORG
3,John Smith agrees to deliver the goods by Marc...,John Smith,PERSON
4,John Smith agrees to deliver the goods by Marc...,"March 10, 2023",DATE


In [81]:
import inspect
from transformers import TrainingArguments
print(inspect.getfile(TrainingArguments))


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py


In [97]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict, ClassLabel, Sequence
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Unique labels
unique_labels = sorted(df["label"].unique())
label_list = ["O"] + [f"B-{l}" for l in unique_labels] + [f"I-{l}" for l in unique_labels]
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

# Create BIO-tagged dataset
def to_bio_format(df):
    token_data = []
    for sentence in df["sentence"].unique():
        sentence_df = df[df["sentence"] == sentence]
        tokens = tokenizer.tokenize(sentence)
        labels = ["O"] * len(tokens)

        for _, row in sentence_df.iterrows():
            entity_tokens = tokenizer.tokenize(row["entity"])
            for i in range(len(tokens) - len(entity_tokens) + 1):
                if tokens[i:i+len(entity_tokens)] == entity_tokens:
                    labels[i] = f"B-{row['label']}"
                    for j in range(1, len(entity_tokens)):
                        labels[i+j] = f"I-{row['label']}"
                    break
        token_data.append({"tokens": tokens, "ner_tags": [label_to_id[l] for l in labels]})
    return token_data

tokenized_data = to_bio_format(df)
dataset = Dataset.from_list(tokenized_data)
dataset = dataset.train_test_split(test_size=0.3)
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 23
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 11
    })
})

In [5]:
import transformers
print(transformers.__version__)
print(transformers.TrainingArguments)

4.55.0
<class 'transformers.training_args.TrainingArguments'>


In [ ]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00


In [7]:
print(transformers.__version__)

4.55.0


In [ ]:
from transformers import TrainingArguments
help(TrainingArguments)

Help on class TrainingArguments in module transformers.training_args:

class TrainingArguments(builtins.object)
 |  TrainingArguments(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.Sch

In [ ]:
%pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=ce255d3535cd4746a6318c641e6a61756aae861adde7fc804a3c8b6103785120
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [83]:
extra_data = [
    # ORG
    {"sentence": "Alpha Corp entered into agreement with Beta LLC on 1st Jan 2021.", "entity": "Alpha Corp", "label": "ORG"},
    {"sentence": "Alpha Corp entered into agreement with Beta LLC on 1st Jan 2021.", "entity": "Beta LLC", "label": "ORG"},
    {"sentence": "Delta Partners signed the contract with Sigma Legal.", "entity": "Delta Partners", "label": "ORG"},
    {"sentence": "Delta Partners signed the contract with Sigma Legal.", "entity": "Sigma Legal", "label": "ORG"},
    {"sentence": "Theta Systems filed a lawsuit against Omega Inc.", "entity": "Theta Systems", "label": "ORG"},
    {"sentence": "Theta Systems filed a lawsuit against Omega Inc.", "entity": "Omega Inc", "label": "ORG"},
    {"sentence": "Agreement was made between Phoenix Group and Lambda Co.", "entity": "Phoenix Group", "label": "ORG"},
    {"sentence": "Agreement was made between Phoenix Group and Lambda Co.", "entity": "Lambda Co", "label": "ORG"},

    # LAW
    {"sentence": "This agreement is governed by the Arbitration Act, 1996.", "entity": "Arbitration Act, 1996", "label": "LAW"},
    {"sentence": "Section 12 of the Information Technology Act applies here.", "entity": "Information Technology Act", "label": "LAW"},
    {"sentence": "The contract is subject to the Companies Act, 2013.", "entity": "Companies Act, 2013", "label": "LAW"},
    {"sentence": "As per Section 3 of the Indian Contract Act, it is valid.", "entity": "Indian Contract Act", "label": "LAW"},
    {"sentence": "This falls under the jurisdiction of the Consumer Protection Act.", "entity": "Consumer Protection Act", "label": "LAW"},

    # DATE
    {"sentence": "The contract was signed on 5th February 2020.", "entity": "5th February 2020", "label": "DATE"},
    {"sentence": "The payment was completed by 31st December 2021.", "entity": "31st December 2021", "label": "DATE"},
    {"sentence": "The case was filed on 10th August 2022.", "entity": "10th August 2022", "label": "DATE"},
    {"sentence": "Final agreement took place on 15th March 2023.", "entity": "15th March 2023", "label": "DATE"},
    {"sentence": "Review was conducted on 20th June 2024.", "entity": "20th June 2024", "label": "DATE"},

    # MONEY
    {"sentence": "The amount transferred was $50,000.", "entity": "$50,000", "label": "MONEY"},
    {"sentence": "An advance of ₹1,00,000 was paid.", "entity": "₹1,00,000", "label": "MONEY"},
    {"sentence": "The settlement amount was USD 10,000.", "entity": "USD 10,000", "label": "MONEY"},
    {"sentence": "They received €5,000 as compensation.", "entity": "€5,000", "label": "MONEY"},
    {"sentence": "The penalty imposed was INR 25,000.", "entity": "INR 25,000", "label": "MONEY"},

    # CLAUSE
    {"sentence": "According to Clause 3 of the agreement, parties must comply.", "entity": "Clause 3", "label": "CLAUSE"},
    {"sentence": "Refer to Section 8(b) for more details.", "entity": "Section 8(b)", "label": "CLAUSE"},
    {"sentence": "Under Clause 5.2, a termination fee is applicable.", "entity": "Clause 5.2", "label": "CLAUSE"},
    {"sentence": "As per Clause 7, all disputes will be arbitrated.", "entity": "Clause 7", "label": "CLAUSE"},
    {"sentence": "Clause 10 outlines the confidentiality obligations.", "entity": "Clause 10", "label": "CLAUSE"},

    # PERSON
    {"sentence": "John Smith signed the contract on behalf of the company.", "entity": "John Smith", "label": "PERSON"},
    {"sentence": "Jane Doe was present during the negotiations.", "entity": "Jane Doe", "label": "PERSON"},
    {"sentence": "The contract was approved by Michael Brown.", "entity": "Michael Brown", "label": "PERSON"},
    {"sentence": "Emily Davis is the legal representative.", "entity": "Emily Davis", "label": "PERSON"},
    {"sentence": "Mr. Thomas agreed to the settlement.", "entity": "Mr. Thomas", "label": "PERSON"},
]


In [84]:
df = pd.concat([df, pd.DataFrame(extra_data)], ignore_index=True)

In [98]:
df.shape

(46, 3)

In [86]:
df.tail()

,sentence,entity,label
41,John Smith signed the contract on behalf of th...,John Smith,PERSON
42,Jane Doe was present during the negotiations.,Jane Doe,PERSON
43,The contract was approved by Michael Brown.,Michael Brown,PERSON
44,Emily Davis is the legal representative.,Emily Davis,PERSON
45,Mr. Thomas agreed to the settlement.,Mr. Thomas,PERSON


In [95]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch

# ======================
# Generate Label Mappings
# ======================
entity_labels = sorted(df["label"].dropna().unique())
label_list = ["O"] + [f"B-{l}" for l in entity_labels] + [f"I-{l}" for l in entity_labels]
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

# ======================
# BIO Conversion
# ======================
def create_examples(df):
    grouped = df.groupby("sentence")
    examples = []

    for sentence, group in grouped:
        tokens = sentence.split()
        labels = ["O"] * len(tokens)

        for _, row in group.iterrows():
            entity_tokens = row["entity"].split()
            label = row["label"]

            for i in range(len(tokens) - len(entity_tokens) + 1):
                if tokens[i:i + len(entity_tokens)] == entity_tokens:
                    labels[i] = f"B-{label}"
                    for j in range(1, len(entity_tokens)):
                        labels[i + j] = f"I-{label}"
                    break

        label_ids = [label_to_id[l] for l in labels]
        examples.append({"tokens": tokens, "ner_tags": label_ids})
    return examples

examples = create_examples(df)

# ======================
# Keep ALL sentences (important for O labels)
# ======================
dataset = Dataset.from_list(examples)
dataset = dataset.train_test_split(test_size=0.3, seed=42)

# ======================
# Tokenizer and Align Labels
# ======================
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        prev_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != prev_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)  # Ignore subwords
            prev_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# ======================
# Load BERT Model
# ======================
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)

# ======================
# Evaluation Metric
# ======================
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=2)

    true_preds = []
    true_labels = []

    for pred, label in zip(preds, labels):
        pred_seq = []
        label_seq = []
        for p_, l_ in zip(pred, label):
            if l_ != -100:
                pred_seq.append(id_to_label[p_])
                label_seq.append(id_to_label[l_])
        true_preds.append(pred_seq)
        true_labels.append(label_seq)

    results = seqeval.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"]
    }

# ======================
# Training Arguments
# ======================
args = TrainingArguments(
    # output_dir="./ner_model",
    eval_strategy="epoch",  # fixed arg name
    save_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none"
)

# ======================
# Trainer
# ======================
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ======================
# Train the Model
# ======================
trainer.train()


Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1712247615.py:143: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.603997,0.000000,0.000000,0.000000,0.881720
2,No log,0.915149,0.000000,0.000000,0.000000,0.881720
3,No log,0.638148,0.000000,0.000000,0.000000,0.881720
4,No log,0.594825,0.000000,0.000000,0.000000,0.881720
5,No log,0.583567,0.000000,0.000000,0.000000,0.881720
6,No log,0.580991,0.000000,0.000000,0.000000,0.881720


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=18, training_loss=1.0313035117255316, metrics={'train_runtime': 3.8669, 'train_samples_per_second': 35.687, 'train_steps_per_second': 4.655, 'total_flos': 9015634626048.0, 'train_loss': 1.0313035117255316, 'epoch': 6.0})

## The above training model got the high accuracy but it couldnt predict the predictions correctly, so I have defined a class called Weights Handler

In [103]:
from datasets import DatasetDict, concatenate_datasets
from collections import Counter
import torch
from torch import nn
from transformers import AutoTokenizer, BertForTokenClassification, Trainer, TrainingArguments
import numpy as np
from seqeval.metrics import classification_report, f1_score

# ✅ 1. Ensure dataset has validation set
if "validation" not in dataset:
    dataset = DatasetDict({
        "train": dataset["train"],
        "validation": dataset["test"]  # rename test -> validation
    })

# --- Assuming label_list, label_to_id, id_to_label are already defined ---

# ✅ 2. Oversample entity-rich examples
def has_entity(example):
    return any(tag != label_to_id["O"] for tag in example["ner_tags"])

train_dataset = dataset["train"]
pos_samples = train_dataset.filter(has_entity)
train_dataset = concatenate_datasets([train_dataset, pos_samples, pos_samples])

dataset["train"] = train_dataset

print("New train size:", len(dataset["train"]))

# ✅ 3. Compute class weights
all_labels = [l for example in dataset["train"]["ner_tags"] for l in example]
counts = Counter(all_labels)
total = sum(counts.values())
weights = [total / counts[i] if counts[i] != 0 else 0.0 for i in range(len(label_list))]
class_weights = torch.tensor(weights, dtype=torch.float)
print("Class Weights:", dict(zip(label_list, weights)))

# ✅ 4. Tokenize & align labels
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["tokens", "ner_tags"])

# ✅ 5. Custom model with weighted loss
class WeightedBertForTokenClassification(BertForTokenClassification):
    def __init__(self, config, class_weights=None):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.class_weights = class_weights
        self.loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        # Remove any keys not accepted by BertModel
        bert_kwargs = {k: v for k, v in kwargs.items() if k in ["position_ids", "head_mask", "inputs_embeds", "encoder_hidden_states", "encoder_attention_mask", "past_key_values", "use_cache", "output_attentions", "output_hidden_states", "return_dict"]}

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **bert_kwargs
        )

        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss = self.loss_fct(
                logits.view(-1, self.num_labels),
                labels.view(-1)
            )

        return {"loss": loss, "logits": logits}


model = WeightedBertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id,
    class_weights=class_weights
)

# ✅ 6. Metrics
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id_to_label[l] for l in label if l != -100]
        for label in labels
    ]
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    return {
        "f1": f1_score(true_labels, true_predictions),
        "report": classification_report(true_labels, true_predictions)
    }

# ✅ 7. Training args (no W&B logging)
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    remove_unused_columns=False,
    report_to=[]  # ⛔ disables W&B
)

# ✅ 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ✅ 9. Train
trainer.train()


Filter:   0%|          | 0/207 [00:00<?, ? examples/s]

New train size: 621
Class Weights: {'O': 1.65, 'B-CLAUSE': 59.4, 'B-DATE': 42.42857142857143, 'B-LAW': 49.5, 'B-MONEY': 99.0, 'B-ORG': 42.42857142857143, 'B-PERSON': 59.4, 'I-CLAUSE': 18.5625, 'I-DATE': 12.91304347826087, 'I-LAW': 14.85, 'I-MONEY': 33.0, 'I-ORG': 33.0, 'I-PERSON': 42.42857142857143}


Map:   0%|          | 0/621 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Some weights of WeightedBertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'loss_fct.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2413236184.py:147: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1,Report
1,1.097600,0.336038,0.645161,precision recall f1-score support CLAUSE 0.67 1.00 0.80 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 0.50 0.67 0.57 3 ORG 0.75 0.75 0.75 4 PERSON 0.33 0.50 0.40 2 micro avg 0.56 0.77 0.65 13 macro avg 0.54 0.82 0.64 13 weighted avg 0.58 0.77 0.65 13
2,0.005000,0.347628,0.733333,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.60 0.75 0.67 4 PERSON 0.33 0.50 0.40 2 micro avg 0.65 0.85 0.73 13 macro avg 0.66 0.88 0.73 13 weighted avg 0.70 0.85 0.75 13
3,0.003300,0.354279,0.733333,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.60 0.75 0.67 4 PERSON 0.33 0.50 0.40 2 micro avg 0.65 0.85 0.73 13 macro avg 0.66 0.88 0.73 13 weighted avg 0.70 0.85 0.75 13
4,0.001900,0.362487,0.733333,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.60 0.75 0.67 4 PERSON 0.33 0.50 0.40 2 micro avg 0.65 0.85 0.73 13 macro avg 0.66 0.88 0.73 13 weighted avg 0.70 0.85 0.75 13
5,0.001600,0.375306,0.733333,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.60 0.75 0.67 4 PERSON 0.33 0.50 0.40 2 micro avg 0.65 0.85 0.73 13 macro avg 0.66 0.88 0.73 13 weighted avg 0.70 0.85 0.75 13
6,0.001200,0.375595,0.733333,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.60 0.75 0.67 4 PERSON 0.33 0.50 0.40 2 micro avg 0.65 0.85 0.73 13 macro avg 0.66 0.88 0.73 13 weighted avg 0.70 0.85 0.75 13
7,0.001000,0.380199,0.733333,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.60 0.75 0.67 4 PERSON 0.33 0.50 0.40 2 micro avg 0.65 0.85 0.73 13 macro avg 0.66 0.88 0.73 13 weighted avg 0.70 0.85 0.75 13
8,0.000800,0.387941,0.758621,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.75 0.75 0.75 4 PERSON 0.33 0.50 0.40 2 micro avg 0.69 0.85 0.76 13 macro avg 0.68 0.88 0.75 13 weighted avg 0.74 0.85 0.78 13
9,0.000700,0.391630,0.733333,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.60 0.75 0.67 4 PERSON 0.33 0.50 0.40 2 micro avg 0.65 0.85 0.73 13 macro avg 0.66 0.88 0.73 13 weighted avg 0.70 0.85 0.75 13
10,0.000700,0.395460,0.758621,precision recall f1-score support CLAUSE 1.00 1.00 1.00 2 DATE 0.50 1.00 0.67 1 LAW 0.50 1.00 0.67 1 MONEY 1.00 1.00 1.00 3 ORG 0.75 0.75 0.75 4 PERSON 0.33 0.50 0.40 2 micro avg 0.69 0.85 0.76 13 macro avg 0.68 0.88 0.75 13 weighted avg 0.74 0.85 0.78 13


TrainOutput(global_step=1560, training_loss=0.037148483269489725, metrics={'train_runtime': 742.2725, 'train_samples_per_second': 16.732, 'train_steps_per_second': 2.102, 'total_flos': 215530015278960.0, 'train_loss': 0.037148483269489725, 'epoch': 20.0})

In [100]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 69
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11
    })
})

In [104]:
# ======================
# STEP 9: Inference Example
# ======================
from transformers import pipeline

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

test_sentence = "Jane Doe signed a contract with Omega Inc on April 15th, 2021."
print(ner_pipeline(test_sentence))


Device set to use cuda:0
The model 'WeightedBertForTokenClassification' is not supported for ner. Supported models are ['PeftModelForTokenClassification', 'AlbertForTokenClassification', 'ArceeForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassification', 'Data2VecTextForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'DiffLlamaForTokenClassification', 'DistilBertForTokenClassification', 'ElectraForTokenClassification', 'ErnieForTokenClassification', 'ErnieMForTokenClassification', 'EsmForTokenClassification', 'Exaone4ForTokenClassification', 'FalconForTokenClassification', 'FlaubertForTokenClassification', 'FNetForTokenClassification', 'FunnelForTokenClassification', 'GemmaForTokenClassification', 'Gemma2ForTokenClassification',

[{'entity_group': 'PERSON', 'score': np.float32(0.93012553), 'word': 'Jane Doe', 'start': 0, 'end': 8}, {'entity_group': 'ORG', 'score': np.float32(0.99966085), 'word': 'Omega Inc', 'start': 32, 'end': 41}, {'entity_group': 'DATE', 'score': np.float32(0.99649876), 'word': 'April 15th, 2021', 'start': 45, 'end': 61}]


In [105]:
from collections import Counter
all_tags = sum([e["ner_tags"] for e in examples], [])
tag_names = [label_list[i] for i in all_tags]
Counter(tag_names)

Counter({'O': 262,
         'B-CLAUSE': 5,
         'I-CLAUSE': 5,
         'B-ORG': 6,
         'I-ORG': 6,
         'B-MONEY': 3,
         'B-PERSON': 5,
         'I-PERSON': 5,
         'B-DATE': 2,
         'I-DATE': 4,
         'B-LAW': 1,
         'I-LAW': 2})

In [106]:
from transformers import pipeline
ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
ner_pipe("Jane Doe of Delta Corp agreed on 5th July 2022 under the Data Act.")

Device set to use cuda:0
The model 'WeightedBertForTokenClassification' is not supported for token-classification. Supported models are ['PeftModelForTokenClassification', 'AlbertForTokenClassification', 'ArceeForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'BioGptForTokenClassification', 'BloomForTokenClassification', 'BrosForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification', 'ConvBertForTokenClassification', 'Data2VecTextForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'DiffLlamaForTokenClassification', 'DistilBertForTokenClassification', 'ElectraForTokenClassification', 'ErnieForTokenClassification', 'ErnieMForTokenClassification', 'EsmForTokenClassification', 'Exaone4ForTokenClassification', 'FalconForTokenClassification', 'FlaubertForTokenClassification', 'FNetForTokenClassification', 'FunnelForTokenClassification', 'GemmaForTokenClassification', 'Gemma2ForToke

[{'entity_group': 'PERSON',
  'score': np.float32(0.9393266),
  'word': 'Jane Doe',
  'start': 0,
  'end': 8},
 {'entity_group': 'ORG',
  'score': np.float32(0.9992541),
  'word': 'Delta Corp',
  'start': 12,
  'end': 22},
 {'entity_group': 'DATE',
  'score': np.float32(0.9892213),
  'word': '5th July 2022',
  'start': 33,
  'end': 46},
 {'entity_group': 'LAW',
  'score': np.float32(0.9992255),
  'word': 'Data Act',
  'start': 57,
  'end': 65}]

In [108]:
from collections import Counter
all_tags = [id_to_label[tag] for example in examples for tag in example["ner_tags"]]
print(Counter(all_tags))

Counter({'O': 262, 'B-ORG': 6, 'I-ORG': 6, 'B-CLAUSE': 5, 'I-CLAUSE': 5, 'B-PERSON': 5, 'I-PERSON': 5, 'I-DATE': 4, 'B-MONEY': 3, 'B-DATE': 2, 'I-LAW': 2, 'B-LAW': 1})


In [110]:
def debug_token_alignment():
    sample = dataset["train"][0]
    tokens = sample["tokens"]
    labels = sample["ner_tags"]

    print("Tokens:", tokens)
    print("Labels:", [id_to_label[l] for l in labels])

debug_token_alignment()

Tokens: ['Jane', 'Do', '##e', 'was', 'present', 'during', 'the', 'negotiations', '.']
Labels: ['B-PERSON', 'I-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O']


In [63]:
# import re
# from datasets import Dataset, DatasetDict

# # Patterns
# ORG_PATTERN = re.compile(r"\b[A-Z][a-zA-Z]+(?:\s(?:Inc|Ltd|Corporation|Corp|LLC))\b")
# DATE_PATTERN = re.compile(r"\b\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{4}\b")
# MONEY_PATTERN = re.compile(r"(?:₹|\$|€)\d+(?:,\d{3})*(?:\.\d{2})?")
# LAW_PATTERN = re.compile(r"\b[A-Z][a-zA-Z]+\sAct,?\s\d{4}\b")  # NEW

# label_list = ['O', 'B-CLAUSE', 'B-DATE', 'B-LAW', 'B-MONEY', 'B-ORG', 'B-PERSON',
#               'I-CLAUSE', 'I-DATE', 'I-LAW', 'I-MONEY', 'I-ORG', 'I-PERSON']
# label_to_id = {l: i for i, l in enumerate(label_list)}

# def pattern_based_labeling(sentence):
#     tokens = sentence.split()
#     labels = ["O"] * len(tokens)

#     def tag_pattern(pattern, label_prefix):
#         for match in pattern.finditer(sentence):
#             match_tokens = match.group(0).split()
#             # Iterate over tokens in match and tag
#             for i, match_word in enumerate(match_tokens):
#                 for idx, tok in enumerate(tokens):
#                     if tok.strip(",.") == match_word.strip(",.") and labels[idx] == "O":
#                         labels[idx] = f"{'B' if i == 0 else 'I'}-{label_prefix}"

#     tag_pattern(ORG_PATTERN, "ORG")
#     tag_pattern(DATE_PATTERN, "DATE")
#     tag_pattern(MONEY_PATTERN, "MONEY")
#     tag_pattern(LAW_PATTERN, "LAW")

#     return tokens, [label_to_id[l] for l in labels]

# # Example data
# data = [
#     {"text": "Omega Inc filed a lawsuit on 15 April 2022."},
#     {"text": "The Arbitration Act, 1996 was enforced."},
#     {"text": "Tesla Corp announced a $5,000 investment."}
# ]

# dataset = Dataset.from_list([
#     {"tokens": pattern_based_labeling(row["text"])[0],
#      "ner_tags": pattern_based_labeling(row["text"])[1]}
#     for row in data
# ])

# # Train-test split
# dataset = dataset.train_test_split(test_size=0.2, seed=42)
# dataset = DatasetDict({
#     "train": dataset["train"],
#     "test": dataset["test"]
# })

# # Debug
# def debug_token_alignment():
#     sample = dataset["train"][0]
#     print("Tokens:", sample["tokens"])
#     print("Labels:", [label_list[l] for l in sample["ner_tags"]])

# debug_token_alignment()


Tokens: ['The', 'Arbitration', 'Act,', '1996', 'was', 'enforced.']
Labels: ['O', 'B-LAW', 'I-LAW', 'I-LAW', 'O', 'O']


In [112]:
from transformers import BertTokenizerFast, BertForTokenClassification
import torch

# 1️⃣ Save model & tokenizer
model_save_path = "./ner_bert_legal"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

# 2️⃣ Load the trained model
model = BertForTokenClassification.from_pretrained(model_save_path)
tokenizer = BertTokenizerFast.from_pretrained(model_save_path)

# Get label mappings from the dataset
id2label = {i: label for i, label in enumerate(label_list)}  # label_list should be from your earlier preprocessing

# 3️⃣ Function to run NER prediction
def predict_ner(sentence):
    # Tokenize
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, is_split_into_words=False)

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)

    # Convert predictions to labels
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [id2label[p.item()] for p in predictions[0]]

    # Merge tokens & labels
    for token, label in zip(tokens, predicted_labels):
        print(f"{token:15} --> {label}")

# 4️⃣ Test on a sentence
test_sentence = "The agreement was signed by John on 5th March 2024."
predict_ner(test_sentence)


[CLS]           --> O
The             --> O
agreement       --> O
was             --> O
signed          --> O
by              --> O
John            --> O
on              --> O
5th             --> B-DATE
March           --> I-DATE
202             --> I-DATE
##4             --> I-DATE
.               --> O
[SEP]           --> O


In [114]:
from transformers import pipeline

# Create an NER pipeline with your fine-tuned model
ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"  # merges subword tokens automatically
)

# Test sentence
sentence = "The agreement was signed by John Smith on 5th March 2024 in accordance with Law 23, and the payment of $5000 was made to Acme Corp"

# Get predictions
predictions = ner_pipeline(sentence)

# Print merged predictions
for entity in predictions:
    word = entity['word']
    label = entity['entity_group']
    score = round(entity['score'], 4)
    print(f"{word:20} --> {label} (score: {score})")


Device set to use cuda:0


John Smith           --> PERSON (score: 0.9988999962806702)
5th March 2024       --> DATE (score: 0.9994000196456909)
Law                  --> CLAUSE (score: 0.9725000262260437)
23                   --> LAW (score: 0.46720001101493835)
$ 5000               --> MONEY (score: 0.9987999796867371)
A                    --> ORG (score: 0.4510999917984009)
##cme Corp           --> ORG (score: 0.7692999839782715)


In [ ]:
from transformers import AutoTokenizer
import torch
import torch.nn.functional as F

# Load saved tokenizer and model directory path
model_path = "./ner_bert_legal"  # your saved model folder path

tokenizer = AutoTokenizer.from_pretrained(model_path)

# If you used a custom model class (like WeightedBertForTokenClassification), 
# you need to define that class first or load with AutoModelForTokenClassification:

from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [5]:
import gradio as gr
import torch
import torch.nn.functional as F
from transformers import BertTokenizerFast, BertForTokenClassification, pipeline

# Path to saved model/tokenizer folder
model_path = "./ner_bert_legal"

# Load model and tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = BertForTokenClassification.from_pretrained(model_path)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define your label map from your training preprocessing, example:
id_to_label = {
    0: "O",
    1: "B-PERSON",
    2: "I-PERSON",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-DATE",
    6: "I-DATE",
    7: "B-CLAUSE",
    8: "I-CLAUSE",
    # Add all your labels here
}

# Create a huggingface pipeline for NER with aggregation
ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

def ner_infer(text):
    try:
        entities = ner_pipeline(text)
        if not entities:
            return "No entities found."

        results = []
        for ent in entities:
            word = ent['word']
            label = ent['entity_group']
            score = ent['score']
            results.append(f"{word}  -->  {label} (score: {score:.4f})")

        return "\n".join(results)
    except Exception as e:
        return f"Error: {str(e)}"

gr.Interface(
    fn=ner_infer,
    inputs=gr.Textbox(lines=4, placeholder="Paste legal sentence here..."),
    outputs="text",
    title="📜 Legal Contract Entity Extractor"
).launch()


Device set to use cpu


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [ ]:
import gradio as gr
import torch
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def ner_infer(text):
    try:
        encoding = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            is_split_into_words=False,
            padding=True
        )
        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask).logits
            probs = F.softmax(outputs, dim=-1)
            preds = torch.argmax(probs, dim=-1)[0].tolist()

        tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        confidences = probs[0, range(len(preds)), preds].tolist()

        entities = []
        current_entity = None
        current_label = None
        current_score_sum = 0
        current_token_count = 0

        special_tokens = set(tokenizer.all_special_tokens)

        for token, pred_id, score in zip(tokens, preds, confidences):
            label = id_to_label[pred_id]

            if token in special_tokens:
                continue

            if label == "O":
                if current_entity:
                    avg_score = current_score_sum / current_token_count
                    entities.append((current_entity, current_label, avg_score))
                    current_entity, current_label = None, None
                continue

            if label.startswith("B-"):
                if current_entity:
                    avg_score = current_score_sum / current_token_count
                    entities.append((current_entity, current_label, avg_score))
                current_entity = token
                current_label = label[2:]
                current_score_sum = score
                current_token_count = 1

            elif label.startswith("I-") and current_entity:
                if token.startswith("##"):
                    current_entity += token[2:]
                else:
                    current_entity += " " + token
                current_score_sum += score
                current_token_count += 1

        if current_entity:
            avg_score = current_score_sum / current_token_count
            entities.append((current_entity, current_label, avg_score))

        highlighted = "\n".join(
            [f"{entity}  -->  {label} (score: {score:.4f})" for entity, label, score in entities]
        )
        return highlighted if highlighted else "No entities found."

    except Exception as e:
        return f"Error: {str(e)}"

gr.Interface(
    fn=ner_infer,
    inputs=gr.Textbox(lines=4, placeholder="Paste legal sentence here..."),
    outputs="text",
    title="📜 Legal Contract Entity Extractor"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a2e631efc8866b9eff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [120]:
import shutil
from google.colab import files
import os

# Get current directory path
current_dir = os.getcwd()

# Zip the current directory (contents inside current dir)
shutil.make_archive('current_dir_backup', 'zip', current_dir)
files.download('current_dir_backup.zip')


'/content/current_dir_backup.zip'

In [123]:
import shutil
from google.colab import files

# Zip only the model folder
shutil.make_archive('ner_bert_legal', 'zip', './ner_bert_legal')

# Download it
files.download('ner_bert_legal.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Using Pretrained NER model

In [ ]:
from transformers import pipeline

# Using dslim/bert-base-NER model--->it doesnt predicts the data entity
# # Load the pretrained NER pipeline model
# ner_pipeline = pipeline(
#     "ner",
#     model="dslim/bert-base-NER",
#     tokenizer="dslim/bert-base-NER",
#     aggregation_strategy="simple"  # Groups tokens into full entities
# )

# # Apply NER to each sentence in your dataframe
# for sentence in df["sentence"].unique():
#     print(f"\nSentence: {sentence}")
#     entities = ner_pipeline(sentence)
#     for entity in entities:
#         print(f"Entity: {entity['word']}, Label: {entity['entity_group']}")
        
# from transformers import pipeline

#using Jean-Baptiste/roberta-large-ner-english
# Load the NER pipeline with a model supporting DATE entities
ner_pipeline = pipeline(
    "ner",
    model="Jean-Baptiste/roberta-large-ner-english",
    tokenizer="Jean-Baptiste/roberta-large-ner-english",
    aggregation_strategy="simple"  # Groups tokens into full entities
)

# Example sentences to test
sentences = [
    "I have a meeting on July 15, 2024.",
    "Barack Obama was born on August 4, 1961.",
    "The conference will be held next Monday."
]

for sentence in sentences:
    print(f"\nSentence: {sentence}")
    entities = ner_pipeline(sentence)
    for entity in entities:
        print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.2f}")


In [ ]:
import gradio as gr
import torch
from transformers import pipeline, BertTokenizerFast, BertForTokenClassification
import pandas as pd

# Load your custom legal model
model_path = "./ner_bert_legal"
tokenizer_custom = BertTokenizerFast.from_pretrained(model_path)
model_custom = BertForTokenClassification.from_pretrained(model_path)
model_custom.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_custom.to(device)

ner_pipeline_custom = pipeline(
    "ner",
    model=model_custom,
    tokenizer=tokenizer_custom,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

# Load the general NER model (dslim)
ner_pipeline_general = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    tokenizer="dslim/bert-base-NER",
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1
)

def normalize_label(label):
    # Map similar labels to a common standard
    if label in ["PER", "PERSON"]:
        return "PERSON"
    if label in ["LOC", "LOCATION"]:
        return "LOCATION"
    if label == "ORG":
        return "ORG"
    if label == "DATE":
        return "DATE"
    if label == "MISC":
        return "MISC"
    if label == "CLAUSE":
        return "CLAUSE"
    if label == "MONEY":
        return "MONEY"
    # Add more mappings if needed
    return label  # fallback to original if no mapping

# combined our trained model with dslim/bert-base-NER model for getting more accurate predictions
def combined_ner(text):
    try:
        ents_custom = ner_pipeline_custom(text)
        ents_custom_filtered = [e for e in ents_custom if normalize_label(e['entity_group']) != 'ORG']

        ents_general = ner_pipeline_general(text)
        ents_general_filtered = ents_general

        combined_entities = []
        seen_spans = set()
        for ent in ents_custom_filtered + ents_general_filtered:
            norm_label = normalize_label(ent["entity_group"])
            span = (ent.get("start", None), ent.get("end", None), norm_label)
            if span not in seen_spans:
                seen_spans.add(span)
                # Overwrite label with normalized label for display consistency
                ent["entity_group"] = norm_label
                combined_entities.append(ent)

        if not combined_entities:
            return "No entities found."

        import pandas as pd
        df = pd.DataFrame([{
            "Entity": ent['word'],
            "Type": ent['entity_group'],
            "Score": f"{ent['score']:.4f}"
        } for ent in combined_entities])

        return df

    except Exception as e:
        return f"Error: {str(e)}"


css = """
#ner-title {
    font-size: 28px;
    font-weight: 700;
    margin-bottom: 10px;
    color: #2a3b5f;
}
#ner-desc {
    font-size: 16px;
    margin-bottom: 20px;
    color: #4a4a4a;
}
.gr-input {
    border-radius: 8px !important;
}
.gr-button {
    background-color: #2a3b5f !important;
    color: white !important;
    font-weight: 600 !important;
    border-radius: 8px !important;
}
"""

with gr.Blocks(css=css, title="NER Extractor") as demo:
    gr.HTML("<h1 id='ner-title'>🧠 Named Entity Recognizer</h1>")
    gr.HTML("<p id='ner-desc'>Paste legal or general text below to extract entities like PERSON, ORG, DATE, CLAUSE, LOCATION, and more.</p>")

    with gr.Row():
        text_input = gr.Textbox(label="Input Text", lines=5, placeholder="Paste your sentence here...")
        output_table = gr.DataFrame(headers=["Entity", "Type", "Score"], interactive=False)

    run_btn = gr.Button("Extract Entities")
    # Removed run_btn.style(full_width=True) because not supported

    run_btn.click(fn=combined_ner, inputs=text_input, outputs=output_table)

demo.launch()
